# Results analysis for proposed VSA and Post-hoc systems

Notebook for the performance analysis of VSA systems on edge device simulation + post-hoc system.

## Post Hoc system

In [ ]:
import pandas as pd
from glob import glob

from tqdm import tqdm

In [ ]:
labels = pd.read_csv('data/HR-Avenue-Labels.csv')

In [ ]:
# Post processing function
def forward_fill(group):
    group = group.copy()
    if pd.isna(group.iloc[0]['AnomalyScore']):
        group.iloc[0, group.columns.get_loc('AnomalyScore')] = 1
    group['AnomalyScore'] = group['AnomalyScore'].ffill()
    return group

def process_preds_to_comparison(results, labels, window_size=100):
    results['vid'] = results['video'].apply(lambda r: r[:2]).astype(int)
    labels['vid'] = labels['vid'].astype(int)
    
    anomaly_preds = results.groupby(['vid', 'frameID']).agg({
        'AnomalyScore': 'min',
    }).reset_index()

    anomaly_preds = anomaly_preds.merge(results[['vid', 'frameID', 'AnomalyScore', 'AnomalyThreshold']], on=['vid', 'frameID', 'AnomalyScore'], how='left')
    anomaly_preds.drop_duplicates(inplace=True)
    comparison = anomaly_preds.merge(labels, how='right', 
                          right_on=['vid', 'Frame_ID'], 
                          left_on=['vid', 'frameID'])
    
    comparison.sort_values(['vid', 'Frame_ID'], inplace=True)
    comparison = comparison.groupby('vid', group_keys=False).apply(forward_fill)
    
    comparison['AnomalyScore'] = comparison['AnomalyScore'].apply(lambda x: 1-x)
    
    comparison['AnomalyScore'] = (
        comparison
        .sort_values(['vid','frameID'])  # ensure correct order
        .groupby(['vid'])['AnomalyScore']
        .transform(lambda x: x.rolling(window=window_size, center=True, min_periods=1).mean())
    )
    
    
    comparison['SmoothedHDAnomaly'] = comparison.apply(lambda row: 1 if row['AnomalyScore'] > 1 - row['AnomalyThreshold'] else 0, axis=1)
    
    return comparison

In [ ]:
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, classification_report, confusion_matrix

aucs = []
for r in sorted(glob('data/results/results_posthocsystem_*[0-9].csv')):
    if 'iso' not in r:
        result= pd.read_csv(r)

        
        iteration = r.split('_')[-1].split('.')[0]

        comparison = process_preds_to_comparison(result, labels, 25)

        auc = roc_auc_score(comparison['Anomaly'], comparison['AnomalyScore'])
        print(f"Iteration: {iteration} AUC: {auc}")

        aucs.append(auc)

In [ ]:
import numpy as np
from scipy import stats

# Parameters
confidence = 0.95
n = len(aucs)
mean = np.mean(aucs)
sem = stats.sem(aucs)  # Standard Error of the Mean

# Confidence interval
h = sem * stats.t.ppf((1 + confidence) / 2, n - 1)
ci_lower, ci_upper = mean - h, mean + h

print(f"Mean: {mean}")
print(f"{confidence*100:.0f}% CI: ({ci_lower:.3f}, {ci_upper:.3f}), Error: {h}")

In [ ]:
confusion_mat_data = process_preds_to_comparison(pd.read_csv('./data/results/results_posthocsystem_' + str(np.argmax(aucs)) + '.csv'), labels, 25)
conf_mat_post_hoc = confusion_matrix(confusion_mat_data['Anomaly'], confusion_mat_data['SmoothedHDAnomaly'])

In [ ]:
false_positive_rate = conf_mat_post_hoc[0,1] / (conf_mat_post_hoc[0,0] + conf_mat_post_hoc[0,1])
false_negative_rate = conf_mat_post_hoc[1,0] / (conf_mat_post_hoc[1,0] + conf_mat_post_hoc[1,1])
false_positive_rate, false_negative_rate

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(confusion_mat_data['Anomaly'], 
                                             confusion_mat_data['SmoothedHDAnomaly']))


In [ ]:
roc_auc_score(confusion_mat_data['Anomaly'], confusion_mat_data['AnomalyScore'])

In [ ]:
confusion_mat_data.to_csv('./data/results/best_posthoc.csv', index=False)

## Edge system performance

In [ ]:
results = []

aucs = []
for r in sorted(glob('./data/results/results_edgesystem_[0-9].csv')):
    if 'iso' not in r:
        result= pd.read_csv(r)

        
        iteration = r.split('_')[-2]

        comparison = process_preds_to_comparison(result, labels, 10)

        auc = roc_auc_score(comparison['Anomaly'], comparison['AnomalyScore'])
        print(f"Iteration: {iteration} AUC: {auc}")

        aucs.append(auc)

In [ ]:
# Parameters
confidence = 0.95
n = len(aucs)
mean = np.mean(aucs)
sem = stats.sem(aucs)  # Standard Error of the Mean

# Confidence interval
h = sem * stats.t.ppf((1 + confidence) / 2, n - 1)
ci_lower, ci_upper = mean - h, mean + h

print(f"Mean: {mean}")
print(f"{confidence*100:.0f}% CI: ({ci_lower:.3f}, {ci_upper:.3f}), Error: {h}")

In [ ]:
conf_mat_data = process_preds_to_comparison(pd.read_csv('./data/results/results_edgesystem_' + str(np.argmax(aucs)) + '.csv'), labels, 25)
conf_mat = confusion_matrix(conf_mat_data['Anomaly'], conf_mat_data['SmoothedHDAnomaly'])

In [ ]:
false_positive_rate = conf_mat[0,1] / (conf_mat[0,0] + conf_mat[0,1])
false_negative_rate = conf_mat[1,0] / (conf_mat[1,0] + conf_mat[1,1])
false_positive_rate, false_negative_rate

In [ ]:
print(classification_report(conf_mat_data['Anomaly'], 
                    conf_mat_data['SmoothedHDAnomaly']))

In [ ]:
conf_mat_data.to_csv('./data/results/best_edge.csv', index=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["font.family"] = "Arial"
plt.rcParams["font.size"] = 14

# Create side-by-side subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# --- Left: Psthoc ---
im = ax1.imshow(conf_mat_post_hoc, interpolation='nearest', cmap=plt.cm.Blues)
ax1.set_title("Posthoc System", fontsize=20)

tick_marks = np.arange(2)
ax1.set_xticks(tick_marks)
ax1.set_xticklabels(["Normal", "Anomaly"], fontsize=16)
ax1.set_yticks(tick_marks)
ax1.set_yticklabels(["Normal", "Anomaly"], fontsize=16)
ax1.set_xlabel("Predicted label", fontsize=18)
ax1.set_ylabel("True label", fontsize=18)

# Add counts and proportions
total = conf_mat_post_hoc.sum()
for i in range(conf_mat_post_hoc.shape[0]):
    for j in range(conf_mat_post_hoc.shape[1]):
        count = conf_mat_post_hoc[i, j]
        prop = count / total
        ax1.text(j, i, f"{count}\n({prop:.2%})",
                 ha="center", va="center",
                 color="white" if count > conf_mat_post_hoc.max()/2 else "black",
                 fontsize=18, fontweight="bold")


# --- Right: Edge Matrix ---
im = ax2.imshow(conf_mat, interpolation='nearest', cmap=plt.cm.Blues)
ax2.set_title("Edge System", fontsize=20)

tick_marks = np.arange(2)
ax2.set_xticks(tick_marks)
ax2.set_xticklabels(["Normal", "Anomaly"], fontsize=16)
ax2.set_yticks(tick_marks)
ax2.set_yticklabels(["Normal", "Anomaly"], fontsize=16)
ax2.set_xlabel("Predicted label", fontsize=18)
ax2.set_ylabel("True label", fontsize=18)

# Add counts and proportions
total = conf_mat.sum()
for i in range(conf_mat.shape[0]):
    for j in range(conf_mat.shape[1]):
        count = conf_mat[i, j]
        prop = count / total
        ax2.text(j, i, f"{count}\n({prop:.2%})",
                 ha="center", va="center",
                 color="white" if count > conf_mat.max()/2 else "black",
                 fontsize=18, fontweight="bold")

fig.tight_layout()
plt.show()

# Isolation Baseline

The baseline isolation forest requires the same analysis

In [ ]:
def process_preds_to_comparison_iso(results, labels, window_size=100):
    results['vid'] = results['video'].apply(lambda r: r[:2]).astype(int)
    labels['vid'] = labels['vid'].astype(int)
    
    anomaly_preds = results.groupby(['vid', 'frameID']).agg({
        'AnomalyScore': 'min',
    }).reset_index()

    anomaly_preds = anomaly_preds.merge(results[['vid', 'frameID', 'AnomalyScore']], on=['vid', 'frameID', 'AnomalyScore'], how='left')
    anomaly_preds.drop_duplicates(inplace=True)
    comparison = anomaly_preds.merge(labels, how='right', 
                          right_on=['vid', 'Frame_ID'], 
                          left_on=['vid', 'frameID'])
    
    comparison.sort_values(['vid', 'Frame_ID'], inplace=True)
    comparison = comparison.groupby('vid', group_keys=False).apply(forward_fill)
    
    comparison['AnomalyScore'] = comparison['AnomalyScore'].apply(lambda x: 1-x)
    
    comparison['AnomalyScore'] = (
        comparison
        .sort_values(['vid','frameID'])  # ensure correct order
        .groupby(['vid'])['AnomalyScore']
        .transform(lambda x: x.rolling(window=window_size, center=True, min_periods=1).mean())
    )
    
        
    return comparison

In [ ]:
preds = pd.read_csv('./data/results/results_isoforest.csv')

comparison = process_preds_to_comparison_iso(preds, labels, window_size=25)

In [ ]:
roc_auc_score(comparison['Anomaly'], comparison['AnomalyScore'])

In [ ]:
comparison.to_csv('data/results/best_isoforest.csv')